<img src="assets/parallelisation.png" align="left" width="600" style="margin-right:15px;"/>

In [ ]:
from dotenv import load_dotenv, find_dotenv
from env_utils import doublecheck_env
from langchain_anthropic import ChatAnthropic

path = find_dotenv()
print("Loaded env from:", path)

# Load environment variables from .env
load_dotenv(find_dotenv())

# Check and print results
doublecheck_env(path)

llm = ChatAnthropic(model="claude-sonnet-4-5-20250929")

In [ ]:
from pydantic import BaseModel, Field
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import display, Image

# Graph state
class State(TypedDict):
    topic: str
    joke: str
    story: str
    poem: str
    combined_output: str


# Nodes
def call_llm_1(state: State):
    """First LLM call to generate initial joke"""

    msg = llm.invoke(f"Write a joke about {state['topic']}")
    return {"joke": msg.content}


def call_llm_2(state: State):
    """Second LLM call to generate story"""

    msg = llm.invoke(f"Write a story about {state['topic']}")
    return {"story": msg.content}


def call_llm_3(state: State):
    """Third LLM call to generate poem"""

    msg = llm.invoke(f"Write a poem about {state['topic']}")
    return {"poem": msg.content}


def aggregator(state: State):
    """Combine the joke and story into a single output"""

    combined = f"Here's a story, joke, and poem about {state['topic']}!\n\n"
    combined += f"STORY:\n{state['story']}\n\n"
    combined += f"JOKE:\n{state['joke']}\n\n"
    combined += f"POEM:\n{state['poem']}"
    return {"combined_output": combined}


# Build workflow
parallel_builder = StateGraph(State)

# Add nodes
parallel_builder.add_node("call_llm_1", call_llm_1)
parallel_builder.add_node("call_llm_2", call_llm_2)
parallel_builder.add_node("call_llm_3", call_llm_3)
parallel_builder.add_node("aggregator", aggregator)

# Add edges to connect nodes
parallel_builder.add_edge(START, "call_llm_1")
parallel_builder.add_edge(START, "call_llm_2")
parallel_builder.add_edge(START, "call_llm_3")
parallel_builder.add_edge("call_llm_1", "aggregator")
parallel_builder.add_edge("call_llm_2", "aggregator")
parallel_builder.add_edge("call_llm_3", "aggregator")
parallel_builder.add_edge("aggregator", END)
parallel_workflow = parallel_builder.compile()

# Show workflow
display(Image(parallel_workflow.get_graph().draw_mermaid_png()))

# Invoke
# state = parallel_workflow.invoke({"topic": "cats"})
#print(state["combined_output"])

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Clear cell output
clear_output(wait=True)

# Create all widgets fresh
topic_input = widgets.Text(
    value='cats',
    placeholder='Enter a topic...',
    description='Topic:',
    continuous_update=False,
    layout=widgets.Layout(width='300px')
)

submit_btn = widgets.Button(
    description='Generate Content',
    button_style='primary',
    icon='check'
)

result_html = widgets.HTML(value='')

def on_button_click(btn):
    """Handle button click"""
    topic = topic_input.value.strip()
    
    if not topic:
        result_html.value = "<p>⚠️ Please enter a topic!</p>"
        return
    
    # Show loading state
    btn.disabled = True
    result_html.value = f"<p>🔄 Generating content about: {topic}...</p>"
    
    try:
        # Invoke workflow
        state = parallel_workflow.invoke({"topic": topic})
        
        # Build HTML string with results
        html = f"""
        <div style="font-family: monospace; padding: 10px;">
            <p><strong>📚 Content about: {topic.title()}</strong></p>
            <hr>
            <p><strong>📖 STORY</strong></p>
            <pre style="white-space: pre-wrap;">{state['story']}</pre>
            <hr>
            <p><strong>😄 JOKE</strong></p>
            <pre style="white-space: pre-wrap;">{state['joke']}</pre>
            <hr>
            <p><strong>✍️ POEM</strong></p>
            <pre style="white-space: pre-wrap;">{state['poem']}</pre>
        </div>
        """
        result_html.value = html
        
    except Exception as e:
        result_html.value = f"<p>❌ Error: {str(e)}</p>"
    
    finally:
        btn.disabled = False

# Attach handler
submit_btn.on_click(on_button_click)

# Display
display(widgets.VBox([
    widgets.HTML("<h3>🎨 Interactive Content Generator</h3>"),
    widgets.HTML("<p>Enter a topic and click 'Generate Content'</p>"),
    widgets.HBox([topic_input, submit_btn]),
    result_html
]))